In [1]:
# env pdal windows
import rasterio
from osgeo import gdal

In [2]:
# merger les tifs de CHM si besoin d'utiliser plusieurs fichiers pour couvrir 1 zone
def mergeTiles(PATH_TO_DIR):
    print("Assemblage d'images tif")
    ortho_irc_list = glob(PATH_TO_DIR + "/*.tif")
    vrt = gdal.BuildVRT(PATH_TO_DIR + "/mergedIRC.vrt", ortho_irc_list)
    gdal.Translate(PATH_TO_DIR + "/merged-chm.tif", vrt, xRes=0.5, yRes=-0.5)
    return True

In [3]:
# List of files to be merged
from pathlib import Path
from glob import glob


PATH_TO_DIR = './lidar_data/image_6'

mergeTiles(PATH_TO_DIR)





Assemblage d'images tif


True

In [4]:
# reprojection 2154 -> 3857
merged_chm = './lidar_data/image_6/merged-chm.tif'
reprojected_chm = './lidar_data/image_6/merged-chm_3857.tif'

from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:3857'

with rasterio.open(merged_chm) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(reprojected_chm, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [6]:

#file_to_cut = './lidar_data/image_2/im2_merged-chm.tif'
file_to_cut = './lidar_data/image_6/merged-chm_3857.tif'
file_reference = '../dataset_4bands/image_6/image_6.tif'
outfile = '../dataset_4bands/image_6/chm_image_6.tif'

In [7]:
# resample chm pour coller avec l'image aerienne (-> meme taille, meme resolution)

import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling

# Open the reference raster to obtain its bounding coordinates and pixel size
with rasterio.open(file_reference) as ref_src:
    left, bottom, right, top = ref_src.bounds
    new_width = ref_src.width
    new_height = ref_src.height
    src_crs = src.crs

    # Define the bounding coordinates for the new window based on the reference raster
    window = from_bounds(left, bottom, right, top, ref_src.transform)

    # Open the raster to be cut
    with rasterio.open(file_to_cut) as src:
        # Read the data within the window with the desired resolution using a sampling method
        data = src.read(window=window, out_shape=(new_height, new_width), resampling=Resampling.bilinear)

        # Update the metadata for the new raster
        new_meta = src.meta
        new_meta.update({"driver": "GTiff",
                         "height": new_height,
                         "width": new_width,
                         "crs" : src_crs ,
                         "transform": ref_src.transform,
                         "nodata": 0
                        
                        })
        
        new_meta.update({"crs" : src_crs })

        # Write the resulting raster to a new file
        with rasterio.open(outfile, "w", **new_meta) as dst:
            dst.write(data)

In [9]:
# corriger le CHM avec nodata et valeurs altitudes negatives -> 0 , h> 10  -> 10
import numpy as np
outfile = '../dataset_4bands/image_6/chm_image_6.tif'

chm_corrected_file = '../dataset_4bands/image_6/chm_image_6_co.tif'
with rasterio.open(outfile) as input_raster:
    # Get the width and height of the input raster
    width, height = input_raster.width, input_raster.height
    
    dst_kwargs = input_raster.meta.copy()
    #dst_kwargs['nodata']= 0.0,  # Specify the data type as float

    # Create an empty output array with the same width and height
    output_array = np.zeros((height, width))
    band1 = input_raster.read(1)
     # Replace 'no data' values with 0
    band1[band1 == input_raster.nodata] = 0.0
    
    output_array[:, :] = band1
    # eleminer les valeurs négatives
    output_array[output_array < 0.01] = 0.0
    # eleminer les valeurs de h > 10m, ctrès rare
    output_array[output_array > 10.01] = 10.0

    # Write the output array to a new raster
    with rasterio.open(chm_corrected_file, 'w', **dst_kwargs) as output_raster:
        #♥output_raster.WriteArray(output_array)
        output_raster.write(output_array, 1)

In [10]:
# verifier si chm decoupé est ok, valeurs corrects et pas nuls
outfile = '../dataset_4bands/image_6/chm_image_6_co.tif'
#outfile = '../dataset_4bands/image_2/image_2.tif'
    
with rasterio.open(outfile) as input_raster:
    kwargs = input_raster.meta
    print(kwargs)

    # Read the first band
    band1 = input_raster.read(1)
    #band4 = input_raster.read(4)
    print(band1.size)

    # Calculate the statistics for the first band
    min_val = np.min(band1)
    max_val = np.max(band1)
    mean_val = np.mean(band1)
    data_type = band1.dtype

    # Print the statistics
    print(f"Minimum value: {min_val}")
    print(f"Maximum value: {max_val}")
    print(f"Mean value: {mean_val}")
    print(f"Data type: {data_type}")

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': 0.0, 'width': 2175, 'height': 4589, 'count': 1, 'crs': CRS.from_epsg(2154), 'transform': Affine(0.2999609655172406, 0.0, 66844.6136,
       0.0, -0.3000190891260531, 5458999.9228)}
9981075
Minimum value: 0.0
Maximum value: 10.008609771728516
Mean value: 0.1548899786158865
Data type: float64
